In [84]:
import pandas as pd
import numpy as np
from my_functions import drop_value

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms

Najpierw trzeba pozbyć się wsystkich tekstowych wartości z bazy, aby model regresji mógł je uwzględnić.

In [52]:
path_pickle = r'E:\python\projects\apartments\pickle'

In [57]:
df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')

drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

In [54]:
drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

In [56]:
# df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_after.pickle')
df_waw.head()

,dzielnica,od,poziom,umeblowanie,zabudowa,powierzchnia,cena całkowita
0,Targówek,Biuro / Deweloper,2,1,Apartamentowiec,60,2301
1,Bielany,Osoby prywatnej,3,1,Apartamentowiec,56,3001
2,Śródmieście,Osoby prywatnej,2,1,Apartamentowiec,29,2201
3,Mokotów,Osoby prywatnej,2,1,Blok,20,2150
4,Wawer,Osoby prywatnej,2,1,Apartamentowiec,60,2800


W kolumnach: "dzielnica", "od", "poziom", "zabudowa", trzeba utworzyć kolumny dummies. Dodatkowo muszę wyrzucić którąś z nowo utworzonych kolumn. W przeciwnym przypadku, model nie będzie w stanie zinterpretować czy zmiana jest spowodowana wartością 1 w tej kolumnie czy 0 w pozostałych.   
Przykład: _Apartamentowiec = 1 - Blok - Dom - Kamienica - Szeregowiec_. Kolumna Apartamentowiec jest idealnie zdefiniowana przez pozostałe 4 kolumny. Prowadzi to do wielu problemów w regresji (nieinterpretowalność coeff, współliniowość zmiennych).   
W kolumnach dummies odrzucam: Mokotów dla "dzielnicy", osoby przywatnej dla "od", powyżej 10 dla "poziom", apartamentowiec dla "zabudowa".

In [70]:
# z pokojami, z umeblowaniem, z poziomami, bez standaryzacji

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
poziom = pd.get_dummies(data=df_waw['poziom'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)
y = df_waw['cena całkowita']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa',
                 'Mokotów', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 512.848562179458
R^2:  0.5729


In [71]:
# z pokojami, bez umeblowania, bez poziomu, bez standaryzacji

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, zabudowa], axis=1)
y = df_waw['cena całkowita']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'zabudowa', 'poziom', 'umeblowanie',
                 'Mokotów', 'Osoby prywatnej', 'Apartamentowiec'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 517.0287890368211
R^2:  0.5659


In [72]:
# z pokojami, bez umeblowania, z poziomami, bez standaryzacji

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
poziom = pd.get_dummies(data=df_waw['poziom'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)
y = df_waw['cena całkowita']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'umeblowanie',
                 'Mokotów', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 512.847581061666
R^2:  0.5729


In [73]:
# bez pokoi, bez umeblowania, z poziomem, bez standaryzacji

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj', 'pokoje'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
poziom = pd.get_dummies(data=df_waw['poziom'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)
y = df_waw['cena całkowita']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'umeblowanie',
                 'Mokotów', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 512.9139887056763
R^2:  0.5728


In [78]:
# cena za pokój, z powierzchnią na pokój, bez standaryzacji

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
df_waw['powierzchnia na pokoj'] = df_waw['powierzchnia'] / df_waw['pokoje']

drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
poziom = pd.get_dummies(data=df_waw['poziom'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)
y = df_waw['cena całkowita'] / df_waw['pokoje']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'umeblowanie', 'powierzchnia',
                 'Mokotów', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 311.0105707458722
R^2:  0.5953


In [82]:
# cena za pokój, z powierzchnią na pokój, ze standaryzacja

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
df_waw['powierzchnia na pokoj'] = df_waw['powierzchnia'] / df_waw['pokoje']

drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
poziom = pd.get_dummies(data=df_waw['poziom'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)
y = df_waw['cena całkowita'] / df_waw['pokoje']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'umeblowanie', 'powierzchnia',
                 'Mokotów', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'], axis=1)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 311.01057074587226
R^2:  0.5953


In [85]:
# cena za pokój, z powierzchnią na pokój, z normalizacja

df_waw = pd.read_pickle(path_pickle + '\df_waw_analyse_before.pickle')
df_waw['powierzchnia na pokoj'] = df_waw['powierzchnia'] / df_waw['pokoje']

drop_value(df_waw, 'zabudowa', 'Loft')
drop_value(df_waw, 'zabudowa', 'Pozostałe')
drop_value(df_waw, 'poziom', 'Suterena')
drop_value(df_waw, 'poziom', 'Poddasze')
df_waw.drop(['cena za pokoj'], axis = 1, inplace = True)

dzielnica = pd.get_dummies(data=df_waw['dzielnica'])
od = pd.get_dummies(data=df_waw['od'])
poziom = pd.get_dummies(data=df_waw['poziom'])
zabudowa = pd.get_dummies(data=df_waw['zabudowa'])

df_waw = pd.concat([df_waw, dzielnica, od, poziom, zabudowa], axis=1)
y = df_waw['cena całkowita'] / df_waw['pokoje']
X = df_waw.drop(['cena całkowita', 'dzielnica', 'od', 'poziom', 'zabudowa', 'umeblowanie', 'powierzchnia',
                 'Mokotów', 'Osoby prywatnej', 'Powyżej 10', 'Apartamentowiec'], axis=1)

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=60)
lm = LinearRegression()
lm.fit(X_train, y_train)
prediction = lm.predict(X_test) 
sqrt = np.sqrt(metrics.mean_squared_error(y_test, prediction))
r_sq = lm.score(X_test, y_test)
print('RMSE:', sqrt)
print('R^2: ', round(r_sq, 4))

RMSE: 311.0105707458722
R^2:  0.5953


 Standaryzacja i normalizacja nic nie zmienia.